In [62]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, tree, preprocessing, metrics
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import RobustScaler
import pandas_profiling
from sklearn.impute import SimpleImputer

In [63]:
projectid = "i-dss-ent-data"

data_frame = pd.read_gbq("""
#query deleted to anonymize data
""",projectid, dialect='standard')



In [64]:
imputerMF = SimpleImputer(strategy="most_frequent")
imputerMean = SimpleImputer(strategy="mean")

In [66]:
data_frame.count()

code    317149
a       317149
b       317149
c       300911
d       317149
e       317149
f       317148
g       158735
h       152803
i       317149
j       317149
k       317149
l       317149
m       317149
n       317149
o       317149
p       317149
q       317149
r       317149
s       317149
t       317149
u       317149
dtype: int64

In [67]:
data_frame2 = data_frame.copy(deep=True)

In [68]:
data_frame2.isnull().sum()

code         0
a            0
b            0
c        16238
d            0
e            0
f            1
g       158414
h       164346
i            0
j            0
k            0
l            0
m            0
n            0
o            0
p            0
q            0
r            0
s            0
t            0
u            0
dtype: int64

In [69]:
data_frame2 = data_frame2.drop(['code'],axis=1)

In [70]:
data_frame2.dtypes

a      int64
b     object
c     object
d      int64
e      int64
f    float64
g     object
h     object
i      int64
j     object
k      int64
l      int64
m     object
n      int64
o      int64
p      int64
q      int64
r    float64
s    float64
t    float64
u     object
dtype: object

In [71]:
data_frame2['b'] = data_frame2['b'].astype(str)
data_frame2['c'] = data_frame2['c'].astype(str)
data_frame2['g'] = data_frame2['g'].astype(str)
data_frame2['h'] = data_frame2['h'].astype(str)
data_frame2['j'] = data_frame2['j'].astype(str)
data_frame2['m'] = data_frame2['m'].astype(str)
data_frame2['u'] = data_frame2['u'].astype(str)

In [72]:
pandas_profiling.ProfileReport(data_frame2)

Number of variables,21
Number of observations,317149
Total Missing (%),0.0%
Total size in memory,50.8 MiB
Average record size in memory,168.0 B
Numeric,11
Categorical,7
Boolean,3
Date,0
Text (Unique),0
Rejected,0


In [73]:
data_frame2.isnull().sum()

a    0
b    0
c    0
d    0
e    0
f    1
g    0
h    0
i    0
j    0
k    0
l    0
m    0
n    0
o    0
p    0
q    0
r    0
s    0
t    0
u    0
dtype: int64

In [74]:
#Imputation of null value using the most common value
data_frame2['f']= imputerMF.fit_transform(data_frame2[['f']])


In [75]:
data_frame2.isnull().sum()

a    0
b    0
c    0
d    0
e    0
f    0
g    0
h    0
i    0
j    0
k    0
l    0
m    0
n    0
o    0
p    0
q    0
r    0
s    0
t    0
u    0
dtype: int64

In [76]:
#get dummies for categorical data
data_frame2 = pd.get_dummies(data_frame2)

In [78]:
data_frame2['r'] = power_transform(data_frame2[['r']],method='box-cox')
data_frame2['s'] = power_transform(data_frame2[['s']],method='yeo-johnson')
data_frame2['i'] = power_transform(data_frame2[['i']],method='box-cox')
data_frame2['t'] = power_transform(data_frame2[['t']],method='box-cox')



In [79]:
data_frame2['q'] = power_transform(data_frame2[['q']],method='yeo-johnson')
data_frame2['o'] = power_transform(data_frame2[['o']],method='yeo-johnson')


In [81]:
X = data_frame2.drop(['a'], axis=1).values
y = data_frame2['a'].values



In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [83]:
feat_labels = data_frame2.drop('a', axis=1).columns
print(feat_labels)

Index(['d', 'e', 'f', 'i', 'k', 'l', 'n', 'o', 'p', 'q', 'r', 's', 't', 'b_CF',
       'b_LC', 'c_None', 'c_OTT', 'c_desktop', 'c_mobile_web_app', 'g_None',
       'g_Originals without Star Trek', 'g_Other Shows', 'g_Sports',
       'g_Star Trek', 'g_Tentpole', 'h_None', 'h_navbar_or_lowerbar',
       'h_upsell_overlay', 'h_video_subscribe_toplay', 'j_no', 'j_yes',
       'm_Desktop', 'm_Mobile Apps', 'm_Mobile Web', 'm_OTT Apps',
       'u_multipleplans', 'u_oneplan'],
      dtype='object')


In [ ]:
forest = RandomForestClassifier(n_estimators=1000,random_state=0,n_jobs=-1)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[f],importances[indices[f]]))
#deleted the result for the sake of anonymizing data

In [84]:
clf_dt = tree.DecisionTreeClassifier(max_depth=10)

In [85]:
clf_dt.fit(X_train,y_train)
clf_dt.score(X_test,y_test)

0.6311419412475695

In [86]:
clf_rf = RandomForestClassifier(n_estimators = 100)

In [87]:
clf_rf.fit(X_train,y_train)#training the model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [88]:
y_pred=clf_rf.predict(X_test)#applying prediction on the test set

In [56]:
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.6035104314467392


In [89]:
clf_gb=ske.GradientBoostingClassifier(n_estimators=50)
clf_gb.fit(X_train,y_train)
y_pred=clf_gb.predict(X_test)


In [90]:
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.6341478795522624
